In [1]:
import os
from skimage.io import imread, imshow
import tensorflow as tf
from preprocessing.preprocessing_factory import get_preprocessing
from nets import nets_factory
%matplotlib inline  

import math
import numpy as np
import sys

from tensorflow.contrib import slim
from datasets import dataset_utils
from datasets.dataset_utils import image_to_tfexample, _dataset_exists, _get_filenames_and_classes, write_label_file, ImageReader, _get_dataset_filename
import pandas as pd
from tqdm import tqdm

In [2]:
MODEL = 'resnet_v2_50'

In [74]:
def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir, tfrecord_filename, _NUM_SHARDS):
  """Converts the given filenames to a TFRecord dataset.
  Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
  """
  assert split_name in ['train', 'validation', 'test']

  num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

  with tf.Graph().as_default():
    image_reader = ImageReader()

    with tf.Session('') as sess:

      for shard_id in range(_NUM_SHARDS):
        output_filename = _get_dataset_filename(
            dataset_dir, split_name, shard_id, tfrecord_filename = tfrecord_filename, _NUM_SHARDS = _NUM_SHARDS)

        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
          start_ndx = shard_id * num_per_shard
          end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
          for i in range(start_ndx, end_ndx):
            sys.stdout.write('\r>> Converting image {}/{} shard {}'.format(i+1, len(filenames), shard_id))
            sys.stdout.flush()

            # Read the filename:
            image_data = tf.gfile.FastGFile(filenames[i], 'rb').read()
            height, width = image_reader.read_image_dims(sess, image_data)
            
            if split_name != 'test':
                class_name = os.path.basename(os.path.dirname(filenames[i]))
                class_id = class_names_to_ids[class_name]
            else:
                img = os.path.splitext(os.path.basename(filenames[i]))[0]
                class_id = int(img[4:])
                
            example = image_to_tfexample(image_data, 'jpg'.encode(), height, width, class_id)
            tfrecord_writer.write(example.SerializeToString())

  sys.stdout.write('\n')
  sys.stdout.flush()

In [6]:
dataset_dir = '/testset'

In [106]:
_convert_dataset(split_name='test', 
                 filenames=filenames, 
                 class_names_to_ids=None, 
                 dataset_dir=dataset_dir, 
                 tfrecord_filename='drivers', 
                 _NUM_SHARDS=40)

>> Converting image 79726/79726 shard 39


In [8]:
def get_split(split_name, dataset_dir, file_pattern=None, file_pattern_for_counting='drivers', items_to_descriptions=None):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later.
    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data
    - file_pattern_for_counting(str): the string name to identify your tfrecord files for counting
    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    # First check whether the split_name is train or validation
    if split_name not in ['train', 'validation', 'test']:
        raise ValueError('split name {} was not recognized.'.format(split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern.format(split_name))

    #Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = file_pattern_for_counting + '_' + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    if len(tfrecords_to_count) == 0:
        raise ValueError('There is no dataset.')
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'image_name': slim.tfexample_decoder.Tensor('image/format')
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
    
    labels_to_names = None
    if split_name != 'test':
        #Create the labels_to_name file
        if dataset_utils.has_labels(dataset_dir):
            labels_to_names = dataset_utils.read_label_file(dataset_dir)

        num_classes = len(labels_to_names)
    else:
        num_classes = None

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        # num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_names = labels_to_names,
        items_to_descriptions = items_to_descriptions)

    return dataset

In [9]:
def load_batch(dataset, batch_size, MODEL, height, width, is_training=True):
    '''
    Loads a batch for training.
    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing
    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).
    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        # common_queue_capacity = 24 + 3 * batch_size,
        common_queue_capacity = 2 * batch_size,
        common_queue_min = 1,
        shuffle=is_training)

    #Obtain the raw image using the get method
    raw_image, label, image_name = data_provider.get(['image', 'label', 'image_name'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    preprocessing_fn = get_preprocessing(MODEL, is_training)
    image = preprocessing_fn(raw_image, height, width)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels, image_names = tf.train.batch(
        [image, raw_image, label, image_name],
        batch_size=batch_size,
        # num_threads = 4,
        num_threads=1,
        # capacity = 4 * batch_size,
        capacity=2 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels, image_names

In [10]:
file_pattern = 'drivers_{}_*.tfrecord'

In [11]:
model = nets_factory.get_network_fn(MODEL, num_classes=10, is_training=False)
image_size = model.default_image_size

In [12]:
#State where your log file is at. If it doesn't exist, create it.
log_dir = '/log/' + MODEL

In [13]:
#Get the latest checkpoint file
checkpoint_file = tf.train.latest_checkpoint(log_dir)

In [14]:
log_eval = 'log_test'

In [17]:
with tf.Graph().as_default() as graph:
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = get_split(split_name='test', 
                        dataset_dir=dataset_dir, 
                        file_pattern=file_pattern, 
                        file_pattern_for_counting='drivers', 
                        items_to_descriptions=None)
    images, image_raw, labels = load_batch(dataset=dataset, 
                                           batch_size=1, 
                                           MODEL=MODEL, 
                                           height=image_size, 
                                           width=image_size, 
                                           is_training=False)
    
    # creat the modellog_eval = 'log_val'
    logits, end_points = model(images)
    
    # define scopes 
    variables_to_restore = slim.get_variables_to_restore()
    # create saver function to restore variables from a checkpoint file
    saver = tf.train.Saver(variables_to_restore)
    def restore_fn(sess):
        return saver.restore(sess, checkpoint_file)
    
#     # create global step for mornitoring
#     global_step = get_or_create_global_step()
#     global_step_op = tf.assign(global_step, global_step+1)
    
    
    # the predictions
    probabilities = end_points['predictions']
    
    sv = tf.train.Supervisor(logdir=log_eval, summary_op=None, saver=None, init_fn=restore_fn)
    
    with sv.managed_session() as sess:
        predictions = [['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']]
        for i in tqdm(range(dataset.num_samples)):
            p, label = sess.run([probabilities, labels])
            p = list(p[0])
            label = ['img_' + str(label[0]) + '.jpg']
            p = label + p
            predictions.append(p)
        pd.DataFrame(predictions).to_csv('submission.csv', index=False, header=False)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /log/resnet_v2_50/model.ckpt-14700


100%|██████████| 79726/79726 [30:42<00:00, 43.27it/s]


In [1]:
import pandas as pd
df = pd.read_csv('submission.csv')

In [2]:
df.head()

img            c0            c1            c2            c3  \
0  img_90892.jpg  1.802470e-05  3.926010e-05  4.444770e-01  9.127220e-06   
1  img_90893.jpg  7.474290e-07  1.908600e-04  4.464620e-05  9.568100e-01   
2  img_90895.jpg  7.835570e-04  1.120520e-04  8.922430e-01  1.058480e-05   
3  img_90896.jpg  1.646080e-05  9.980970e-01  7.735730e-06  4.388340e-07   
4  img_90897.jpg  4.001660e-02  5.166600e-10  5.264030e-08  2.206080e-04   

         c4            c5            c6            c7        c8        c9  
0  0.000481  9.475340e-03  1.116580e-02  3.551730e-01  0.146571  0.032591  
1  0.040360  4.195450e-07  1.032940e-05  7.093470e-05  0.002456  0.000056  
2  0.000004  4.476550e-06  1.463100e-04  9.266360e-04  0.105593  0.000176  
3  0.000002  5.476100e-07  5.327090e-05  5.093550e-07  0.001791  0.000031  
4  0.046487  6.294720e-07  1.306170e-08  2.108920e-03  0.000934  0.910233

In [7]:
df = df.sort_values(by='img', ascending=1)
df.head()

img            c0            c1            c2            c3  \
69756       img_1.jpg  1.497460e-01  9.004680e-06  2.210470e-05  3.267790e-05   
69757      img_10.jpg  1.727530e-04  7.866160e-06  1.249400e-05  1.528170e-05   
69758     img_100.jpg  9.711780e-01  9.697410e-03  1.700450e-04  2.508670e-03   
69759    img_1000.jpg  1.462590e-08  9.708770e-10  5.859830e-09  6.297600e-11   
69760  img_100000.jpg  5.994230e-05  2.677400e-06  3.409450e-06  9.971320e-01   

                 c4            c5        c6            c7        c8        c9  
69756  3.598390e-05  2.834180e-01  0.000958  5.957070e-03  0.268042  0.291779  
69757  5.472760e-06  8.521920e-01  0.002430  2.385580e-04  0.144901  0.000025  
69758  1.154090e-02  1.296860e-05  0.000024  1.359090e-03  0.000977  0.002532  
69759  1.315720e-09  1.369930e-07  0.000022  2.694590e-05  0.999590  0.000362  
69760  1.511960e-04  8.694790e-06  0.002134  5.183420e-07  0.000500  0.000008

In [9]:
df.to_csv('submission1.csv', index=False)